In [1]:
extraction_path = r'D:\ADP\Data_Extraction\data_extraction\src\runs\4cfa92fc-d37e-4d2a-a157-2bf351975ee4'

In [20]:
import json
import pandas as pd
from pathlib import Path

def json_to_markdown(json_path, cols=2, spacer=True, sort_keys=False):
    """
    Render one JSON (with Form_fields and Tables) into Markdown.
    - cols: number of Field/Value pairs per row in the Form Fields section
    - spacer: insert a blank spacer column between pairs
    - sort_keys: alphabetically sort Form_fields keys for stable layout
    Returns the Markdown string.
    """
    data = json.loads(Path(json_path).read_text())

    # ---------- Form Fields as a wide table ----------
    items = list((data.get('Form_fields') or {}).items())
    if sort_keys:
        items = sorted(items, key=lambda kv: kv.lower())

    # Build column headers with unique names for pandas/Markdown
    headers = []
    for c in range(cols):
        suffix = '' if c == 0 else ' ' * c
        headers += [f'Field{suffix}', f'Value{suffix}']
        if spacer and c < cols - 1:
            headers.append(' ' * (c + 1))  # unique blank spacer names

    # Rows combining `cols` pairs per row
    rows = []
    for i in range(0, len(items), cols):
        row = {h: '' for h in headers}
        for c in range(cols):
            idx = i + c
            if idx < len(items):
                k, v = items[idx]
                suffix = '' if c == 0 else ' ' * c
                row[f'Field{suffix}'] = k
                row[f'Value{suffix}'] = '' if v is None else str(v)
        rows.append(row)

    ff_df = pd.DataFrame(rows, columns=headers)

    parts = ['## Form Fields', ff_df.to_markdown(index=False), '\n## Tables']

    # ---------- Tables (pipe-separated via pandas) ----------
    for name, trows in (data.get('Tables') or {}).items():
        df = pd.DataFrame(trows)
        parts.append(f"\n### {name}")
        parts.append(df.to_markdown(index=False) if not df.empty else '_(empty_)')

    return '\n'.join(parts)

import os
extraction_path = r'D:\ADP\Data_Extraction\data_extraction\src\runs\4cfa92fc-d37e-4d2a-a157-2bf351975ee4'
base = Path(extraction_path)

for page_dir in os.listdir(base):    
    out_path = Path(base / page_dir / 'original_extraction.md')
    json_path = base / page_dir / 'original_extraction.json'
    md = json_to_markdown(json_path, cols=3, spacer=True, sort_keys=False)
    out_path.write_text(md)
    out_path = Path(base / page_dir / 'corrected_extraction.md')
    json_path = base / page_dir / 'corrected_extraction.json'
    md = json_to_markdown(json_path, cols=3, spacer=True, sort_keys=False)
    out_path.write_text(md)
    #print(md)

In [3]:
import os
from pathlib import Path
import json
extraction_path = r'D:\ADP\Data_Extraction\data_extraction\src\runs\4cfa92fc-d37e-4d2a-a157-2bf351975ee4'
base = Path(extraction_path)

extracted_data = []
for page_dir in os.listdir(base):  
    extracted_data.append(json.loads(Path(fr'{extraction_path}\{page_dir}\corrected_extraction.json').read_text()))

In [4]:
len(extracted_data)

10

In [5]:
extracted_data[0]

{'Form_fields': {'Company Name': 'Velorynt Labs',
  'Company Code': '37546',
  'Period': '12/17/2024 to 12/26/2024',
  'Employee Name': 'Caroline Jones',
  'Address line 1': 'Stream Apt. 219',
  'City': 'Riverton',
  'State': 'UT',
  'Zip': '47589',
  'Emp Id': '4632',
  'SSN': '088-39-6286',
  'DOB': '12/26/2001',
  'Gender': 'M',
  'Marital Status': 'S',
  'Status': 'A',
  'Position': '',
  'Title': '',
  'Pay Group': '',
  'Job Code': '',
  'Emp Type': '',
  'Statutory': '0.00',
  'Seasonal': '0.00',
  'Domestic Emp': 'No',
  'Probation': '0.00',
  'Home #': '509-121-3247',
  'Work #': '',
  'Ext.': '',
  'Email': '',
  'Mail Stop': '',
  'Hire Date': '04/19/2023',
  'Rehire Date': '',
  'Term Date': '',
  'Term Reason': '',
  'Adj Sen Date': '',
  'Pension': 'No',
  'Visa Type': '',
  'Exp': '',
  'Citizen': '',
  'I9 Reverify': '',
  'I9 Verified': 'No',
  'Deceased': 'No',
  'Tax Form': 'W2',
  'WCC': '8810',
  'EEOC': '',
  'Supervisor ID': '',
  'Name (supervisor name)': '',
  

In [8]:
extracted_data[1]

{'Form_fields': {'Company Name': 'Velorynt Labs',
  'Company Code': '37546',
  'Period': '12/17/2024 to 12/26/2024',
  'Employee Name': '',
  'Address line 1': '',
  'City': '',
  'State': '',
  'Zip': '',
  'Emp Id': '',
  'SSN': '',
  'DOB': '',
  'Gender': '',
  'Marital Status': '',
  'Status': '',
  'Position': '',
  'Title': '',
  'Pay Group': '',
  'Job Code': '',
  'Emp Type': '',
  'Statutory': '',
  'Seasonal': '',
  'Domestic Emp': '',
  'Probation': '',
  'Home #': '',
  'Work #': '',
  'Ext.': '',
  'Email': '',
  'Mail Stop': '',
  'Hire Date': '',
  'Rehire Date': '',
  'Term Date': '',
  'Term Reason': '',
  'Adj Sen Date': '',
  'Pension': '',
  'Visa Type': '',
  'Exp': '',
  'Citizen': '',
  'I9 Reverify': '',
  'I9 Verified': '',
  'Deceased': '',
  'Tax Form': '',
  'WCC': '',
  'EEOC': '',
  'Supervisor ID': '',
  'Name (supervisor name)': '',
  'Def Comp': '',
  'Union': '',
  'Union Date': '',
  'Collect Dues': '',
  'Paid Init. Fees': '',
  'Veteran': '',
  'Le

In [6]:
merged_emp_data = [extracted_data[0]]

for emp_data in extracted_data[1:]:
    if 'Emp Id' in emp_data.keys() and not emp_data['Emp Id']:
        tables_to_merge = [tab for tab in emp_data['Tables'] if tab]
        
        merged_emp_data['Tables'].update

In [7]:
from copy import deepcopy

IDENT_KEYS = ["Emp Id"]

# Per-table preferred key fields (use any that exist in a given row)
ROW_KEYS = {
    "Rate/Salary Information": ["RateCode", "Effective Dates"],
    "Employee Tax Information": ["Employee Tax Code", "Effective Dates"],
    "Employer Tax Information": ["Employer Tax Code", "Effective Dates"],
    "Deduction Information": ["Code", "Effective Dates"],
    "Direct Deposit Information": ["Transit No.", "Account No.", "Sequence No."],
    "Fringe Benefit Information": ["ECode", "Effective Dates"],
    "Benefit Accrual Information": ["BCode", "Effective Dates"],
    "Review Information": [],  # fallback to whole-row tuple if empty
    "Emergency Contact Information": ["Name", "Relationship", "Home Phone"],
}

def has_identifier(form_fields: dict) -> bool:
    return any(str(form_fields.get(k, "")).strip() for k in IDENT_KEYS)

def merge_scalar(a_val, b_val, prefer="keep_existing"):
    # prefer: "keep_existing" keeps a_val if non-empty; "prefer_new" prefers b_val
    def is_empty(v):
        return v is None or (isinstance(v, str) and v.strip() == "")
    if prefer == "prefer_new":
        return b_val if not is_empty(b_val) else a_val
    # keep_existing
    return a_val if not is_empty(a_val) else b_val

def deep_merge_dicts(a: dict, b: dict, prefer="keep_existing") -> dict:
    out = deepcopy(a)
    for k, v in b.items():
        if k in out and isinstance(out[k], dict) and isinstance(v, dict):
            out[k] = deep_merge_dicts(out[k], v, prefer=prefer)
        else:
            out[k] = merge_scalar(out.get(k), v, prefer=prefer)
    return out

def row_key(table_name: str, row: dict):
    keys = ROW_KEYS.get(table_name, [])
    parts = []
    for k in keys:
        if k in row and str(row[k]).strip():
            parts.append((k, str(row[k]).strip()))
    if parts:
        return (table_name, tuple(parts))
    # Fallback: stable tuple of sorted items to avoid raw JSON hashing issues
    return (table_name, tuple(sorted((k, str(v)) for k, v in row.items())))

def merge_rows(table_name: str, rows_a: list, rows_b: list, prefer="prefer_new") -> list:
    # Build map keyed by composite key; merge duplicates field-by-field
    by_key = {}
    order = []

    def add_or_merge(row, from_b=False):
        k = row_key(table_name, row)
        if k not in by_key:
            by_key[k] = deepcopy(row)
            order.append(k)
            return
        # Merge field-by-field: fill blanks, optionally prefer_new to override
        current = by_key[k]
        for fk, fv in row.items():
            current[fk] = merge_scalar(current.get(fk), fv, prefer=("prefer_new" if from_b else "keep_existing"))

    for r in rows_a:
        add_or_merge(r, from_b=False)
    for r in rows_b:
        add_or_merge(r, from_b=True)

    return [by_key[k] for k in order]

def merge_tables(tables_a: dict, tables_b: dict) -> dict:
    all_tables = set(tables_a.keys()) | set(tables_b.keys())
    out = {}
    for t in all_tables:
        a_rows = tables_a.get(t, []) or []
        b_rows = tables_b.get(t, []) or []
        if isinstance(a_rows, list) and isinstance(b_rows, list):
            out[t] = merge_rows(t, a_rows, b_rows, prefer="prefer_new")
        else:
            # Non-list tables: fall back to deep dict merge
            out[t] = deep_merge_dicts(a_rows or {}, b_rows or {}, prefer="prefer_new")
    return out

def merge_employee_pages(pages: list[dict]) -> list[dict]:
    merged_records = []
    current = None

    for page in pages:
        form_fields = page.get("Form_fields", {}) or {}
        tables = page.get("Tables", {}) or {}

        if has_identifier(form_fields) or current is None:
            # Start a new record
            current = {
                "Form_fields": deepcopy(form_fields),
                "Tables": deepcopy(tables),
            }
            merged_records.append(current)
        else:
            # Continuation: merge into current record
            current["Form_fields"] = deep_merge_dicts(
                current.get("Form_fields", {}), form_fields, prefer="keep_existing"
            )
            current["Tables"] = merge_tables(current.get("Tables", {}), tables)

    return merged_records

# Example usage:
# pages = [page1_json, page2_json, ...]



In [8]:
result = merge_employee_pages(extracted_data)
print(result)

[{'Form_fields': {'Company Name': 'Velorynt Labs', 'Company Code': '37546', 'Period': '12/17/2024 to 12/26/2024', 'Employee Name': 'Caroline Jones', 'Address line 1': 'Stream Apt. 219', 'City': 'Riverton', 'State': 'UT', 'Zip': '47589', 'Emp Id': '4632', 'SSN': '088-39-6286', 'DOB': '12/26/2001', 'Gender': 'M', 'Marital Status': 'S', 'Status': 'A', 'Position': '', 'Title': '', 'Pay Group': '', 'Job Code': '', 'Emp Type': '', 'Statutory': '0.00', 'Seasonal': '0.00', 'Domestic Emp': 'No', 'Probation': '0.00', 'Home #': '509-121-3247', 'Work #': '', 'Ext.': '', 'Email': '', 'Mail Stop': '', 'Hire Date': '04/19/2023', 'Rehire Date': '', 'Term Date': '', 'Term Reason': '', 'Adj Sen Date': '', 'Pension': 'No', 'Visa Type': '', 'Exp': '', 'Citizen': '', 'I9 Reverify': '', 'I9 Verified': 'No', 'Deceased': 'No', 'Tax Form': 'W2', 'WCC': '8810', 'EEOC': '', 'Supervisor ID': '', 'Name (supervisor name)': '', 'Def Comp': 'No', 'Union': '', 'Union Date': '', 'Collect Dues': 'No', 'Paid Init. Fees':

In [9]:
len(result)

6

In [10]:
result[0]

{'Form_fields': {'Company Name': 'Velorynt Labs',
  'Company Code': '37546',
  'Period': '12/17/2024 to 12/26/2024',
  'Employee Name': 'Caroline Jones',
  'Address line 1': 'Stream Apt. 219',
  'City': 'Riverton',
  'State': 'UT',
  'Zip': '47589',
  'Emp Id': '4632',
  'SSN': '088-39-6286',
  'DOB': '12/26/2001',
  'Gender': 'M',
  'Marital Status': 'S',
  'Status': 'A',
  'Position': '',
  'Title': '',
  'Pay Group': '',
  'Job Code': '',
  'Emp Type': '',
  'Statutory': '0.00',
  'Seasonal': '0.00',
  'Domestic Emp': 'No',
  'Probation': '0.00',
  'Home #': '509-121-3247',
  'Work #': '',
  'Ext.': '',
  'Email': '',
  'Mail Stop': '',
  'Hire Date': '04/19/2023',
  'Rehire Date': '',
  'Term Date': '',
  'Term Reason': '',
  'Adj Sen Date': '',
  'Pension': 'No',
  'Visa Type': '',
  'Exp': '',
  'Citizen': '',
  'I9 Reverify': '',
  'I9 Verified': 'No',
  'Deceased': 'No',
  'Tax Form': 'W2',
  'WCC': '8810',
  'EEOC': '',
  'Supervisor ID': '',
  'Name (supervisor name)': '',
  

In [11]:
for emp_data in result:
    emp_id = emp_data['Form_fields']['Emp Id']
    tables = emp_data['Tables']
    for table in tables.keys():
        for row in tables[table]:
            row['Emp Id'] = emp_id

In [12]:
result[0]['Tables']

{'Direct Deposit Information': [{'Sequence No.': '99.00',
   'Transit No.': '296075933',
   'Account No.': '2294694696',
   'Checking?': 'Yes',
   'Account Name': 'Caroline',
   'Amount Code': '%',
   'Amount': '100.00',
   'Prenote Date': '04/28/2023',
   'Effective Dates': '04/01/2023 to 12/31/2100',
   'Exclude Special': 'No',
   'Emp Id': '4632'}],
 'Emergency Contact Information': [{'Name': 'Alexa Taylor',
   'Relationship': 'Partner',
   'Home Phone': '276-237-7575',
   'Work Phone': '',
   'Address': '992 Maria Plain',
   'City': 'Jamesborough',
   'State': 'PR',
   'Zip': '52030',
   'Country': '',
   'Emp Id': '4632'},
  {'Name': 'Nicole Miller',
   'Relationship': 'Brother',
   'Home Phone': '149-629-1234',
   'Work Phone': '',
   'Address': '2737 Sexton Glens',
   'City': 'Thomasport',
   'State': 'PR',
   'Zip': '55119',
   'Country': '',
   'Emp Id': '4632'}],
 'Review Information': [],
 'Tax Information (Employer)': [{'Employer Tax (code + description)': 'MED-R Medicare -

In [13]:
import pandas as pd

In [14]:
MF = pd.DataFrame({k: [v] for k,v in result[0]['Form_fields'].items()})

for res in result[1:]:
    MF.loc[len(MF)] = res['Form_fields']

In [15]:
MF

,Company Name,Company Code,Period,Employee Name,Address line 1,City,State,Zip,Emp Id,SSN,...,surname,Prior Last,Disability,Smoker,AutoPay,Pay Frequency,OT Exempt,Default Hours,Locations,Positions
0,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Caroline Jones,Stream Apt. 219,Riverton,UT,47589,4632,088-39-6286,...,,,,No,,B,No,0.00,605,700
1,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Jeffrey Bennett,3407 French River,Juliemouth,ND,55428,2796,795-56-8275,...,,,,No,,B,No,0.00,605,700
2,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Jocelyn Taylor,1006 Shery St,Shaontown,WY,93506-7144,5532,609-85-2596,...,,,,No,,B,No,0.00,606,700
3,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Joel Dorsey,1420 Brookview Lane,Clarkville,MA,76911,1121,808-03-9317,...,,,,No,,B,No,0.00,606,700
4,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Patricia Paul,515 Brittany Forks,Savannah,GA,59966,3602,145-16-8120,...,,,,No,,B,No,0.00,606,700
5,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Laura Smith,8356 Joy Lakes,Debrabury,FL,26024,5908,351-20-2239,...,,,,No,,B,No,0.00,606,700


In [16]:
import pandas as pd

# Optional: faster MF build (instead of row-by-row append)
MF = pd.DataFrame([rec.get('Form_fields', {}) for rec in result])

# Choose which Form_fields to carry into every table row
CTX_COLS = ["Emp Id", "Employee Name"]

tables_rows = {}  # table_name -> list of row dicts

for rec in result:
    ff = rec.get("Form_fields", {}) or {}
    ctx = {k: ff.get(k) for k in CTX_COLS}
    for tname, rows in (rec.get("Tables") or {}).items():
        if not isinstance(rows, list):
            continue
        bucket = tables_rows.setdefault(tname, [])
        for r in rows:
            # merge context with the row
            bucket.append({**ctx, **(r or {})})

# Build one DataFrame per table
table_dfs = {t: pd.DataFrame(rows) for t, rows in tables_rows.items()}

# Optional: put context columns first
for t, df in table_dfs.items():
    ctx_first = [c for c in CTX_COLS if c in df.columns]
    other = [c for c in df.columns if c not in ctx_first]
    table_dfs[t] = df.reindex(columns=ctx_first + other)


In [17]:
MF

,Company Name,Company Code,Period,Employee Name,Address line 1,City,State,Zip,Emp Id,SSN,...,surname,Prior Last,Disability,Smoker,AutoPay,Pay Frequency,OT Exempt,Default Hours,Locations,Positions
0,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Caroline Jones,Stream Apt. 219,Riverton,UT,47589,4632,088-39-6286,...,,,,No,,B,No,0.00,605,700
1,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Jeffrey Bennett,3407 French River,Juliemouth,ND,55428,2796,795-56-8275,...,,,,No,,B,No,0.00,605,700
2,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Jocelyn Taylor,1006 Shery St,Shaontown,WY,93506-7144,5532,609-85-2596,...,,,,No,,B,No,0.00,606,700
3,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Joel Dorsey,1420 Brookview Lane,Clarkville,MA,76911,1121,808-03-9317,...,,,,No,,B,No,0.00,606,700
4,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Patricia Paul,515 Brittany Forks,Savannah,GA,59966,3602,145-16-8120,...,,,,No,,B,No,0.00,606,700
5,Velorynt Labs,37546,12/17/2024 to 12/26/2024,Laura Smith,8356 Joy Lakes,Debrabury,FL,26024,5908,351-20-2239,...,,,,No,,B,No,0.00,606,700


In [18]:
for tab in table_dfs.keys():
    print(table_dfs[tab])

  Emp Id    Employee Name Sequence No. Transit No. Account No. Checking?  \
0   4632   Caroline Jones        99.00   296075933  2294694696       Yes   
1   2796  Jeffrey Bennett        99.00   091000022  7301097572       Yes   
2   5532   Jocelyn Taylor        99.00   296076262  2743046665       Yes   
3   1121      Joel Dorsey        99.00   296076301  5589890010       Yes   
4   3602    Patricia Paul        99.00   291070001  8224461774       Yes   
5   5908      Laura Smith        99.00   091000019  9414336673       Yes   

      Account Name Amount Code  Amount Prenote Date           Effective Dates  \
0         Caroline           %  100.00   04/28/2023  04/01/2023 to 12/31/2100   
1         Jefferey           %  100.00   01/01/2015  01/01/2000 to 12/31/2100   
2  Jocelyn, Taylor           %  100.00   02/12/2016  02/12/2016 to 12/31/2100   
3      Joel Dorsey           %  100.00   07/31/2023  07/31/2023 to 12/31/2100   
4   Patricia, Paul           %  100.00   06/30/2017  06/30/201

In [19]:
import re
import pandas as pd

def sanitize_sheet_name(name: str) -> str:
    # Excel invalid chars: : \ / ? * [ ]
    name = re.sub(r'[:\\/?*\[\]]', '_', str(name)).strip()
    # Excel sheet name limit: 31 chars
    return name[:31] if len(name) > 31 else name or "Sheet"

def unique_sheet_name(base: str, used: set) -> str:
    name = sanitize_sheet_name(base)
    if name not in used:
        used.add(name)
        return name
    # Deduplicate with numeric suffixes
    i = 1
    while True:
        suffix = f"_{i}"
        # ensure total <= 31
        trimmed = name[: max(0, 31 - len(suffix))] + suffix
        if trimmed not in used:
            used.add(trimmed)
            return trimmed
        i += 1

# MF: DataFrame of Form_fields
# table_dfs: dict[str, DataFrame] mapping table name -> DataFrame

output_path = "4cfa92fc-d37e-4d2a-a157-2bf351975ee4.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    # Write master form fields
    mf_sheet = unique_sheet_name("Form_fields", used=set())
    MF.to_excel(writer, sheet_name=mf_sheet, index=False)

    used = {mf_sheet}
    # Write each table DataFrame to its own sheet
    for tname, df in table_dfs.items():
        sname = unique_sheet_name(tname, used)
        df.to_excel(writer, sheet_name=sname, index=False)

        # Optional: basic formatting with xlsxwriter (freeze header + auto width)
        ws = writer.sheets[sname]
        # Freeze top row (row index 1 means freeze above row 1 -> first data row)
        ws.freeze_panes(1, 0)
        # Set column widths based on max of header/value lengths (cap for safety)
        for col_idx, col in enumerate(df.columns):
            header_len = len(str(col))
            series = df[col].astype(str)
            value_len = series.map(len).replace({float("nan"): 0}).max() if not df.empty else 0
            width = min(max(header_len, int(value_len)), 60) + 2
            ws.set_column(col_idx, col_idx, width)
